In [2]:
## UI
# general imports
import os
import logging
import numpy as np
import time
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import constants


# models
from src.models import t2i_recsys
from src.models.avg_embeddings_model import AvgEmbeddings
from src.models.soft_cosine_model import SoftCosine
from src.models.knn_model import KNN
# from src.models.USE_model import USE_Recsys
from src.nlp_util.textacy_util import extract_textrank_from_text

# API and UI files
from src import api_helper
from src import tagging_api

In [3]:
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import constants

In [4]:
import pandas as pd
import numpy as np

#get training path
train_dir = constants.TRAIN_DIR
clean_dir = constants.CLEAN_DIR
art_prefix = constants.Text_Prefix
img_prefix = constants.Media_Prefix
#tag types
tag_types = ['org', 'place', 'subject','person']
# article_summary = pd.read_csv(f'{train_dir}/{art_prefix}summary.csv')
article_summary = pd.read_csv(f'{constants.CLEAN_DIR}/article_summary.csv')
image_summary =  pd.read_csv(f'{train_dir}/{img_prefix}summary.csv')
#preview dir
preview_dir = f'{constants.DATA_DIR}/preview'

In [ ]:
title = article_summary.title[0]
body = article_summary.full_text[0]#.values[0]

In [27]:
sample = pd.read_csv("/Users/pwong/Downloads/data_sample.csv")

In [12]:
title = article_summary.title[0]
body = article_summary.full_text[0]#.values[0]

In [99]:
y_true = sample['true_imgs'][0]
y_pred = sample['knn'][0]

In [105]:
def clean_up_true(y):
    y = y.replace('[', '').replace(']', '').replace('"', '').strip().split()
    y = [b.replace('"', '').strip()[1:-1] for b in y]
    return y

In [110]:
def clean_up_pred(y):
    y = y.replace('[', '').replace(']', '').replace('"', '').strip().split(',')
    y = [b.replace('"', '').strip()[1:-1] for b in y]
    return y

In [111]:
y_pred = clean_up_pred(y_pred)
y_true = clean_up_true(y_true)

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [132]:
y_true = ['a','b']
y_pred = ['a', 'c', 'd']

In [22]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

def recall(y_true, y_pred):
    num_true_pos = len(intersection(y_true, y_pred)) # number of true positive
    num_true_image = len(y_true) # number of true images used in the article
    score = num_true_pos/num_true_image
    return score

def precision(y_true, y_pred):
    num_true_pos = len(intersection(y_true, y_pred)) # number of true positive
    num_pred = len(y_pred) # number of predicted images
    score = num_true_pos/num_pred 
    return score

In [34]:
def r_precision(y_true, y_pred):
    num_true_image = len(y_true) # number of true images used in the article
    num_true_pos = len(intersection(y_true, y_pred[:num_true_image])) # number of true positive
    
    score = num_true_pos/num_true_image
    return score    

In [170]:
def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(y_true, y_pred, k=len(y_true), method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    r = [1* (true_img in y_pred[:len(y_true)]) for true_img in y_true] # 1 if in true images
    print("r", r)
    dcg_max = dcg_at_k(np.ones(len(y_true)), k = k) # IDCG with ground truth 
    print('IDCG:', dcg_max)
#     dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max